In [1]:
from bs4 import BeautifulSoup as bs
import requests
import re
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [2]:
def convert_price_to_float(price_str):
    # Check if the price_str is valid
    if not price_str or '----' in price_str:
        return None
    try:
        # Remove the currency symbol
        price_str = price_str.replace('R$', '').strip()
        # Remove the thousands separator (period)
        price_str = price_str.replace('.', '')
        # Replace comma with period for decimal
        price_str = price_str.replace(',', '.')
        # Convert to float
        price_float = float(price_str)
        return price_float
    except ValueError:
        # Return None if conversion fails
        return None

In [19]:
URL = 'https://www.kabum.com.br/hardware/disco-rigido-hd?page_number=1&page_size=100&facet_filters=eyJJbnRlcmZhY2UiOlsiU0FUQSJdfQ==&sort=most_searched'

r = requests.get(URL)
soup = bs(r.content)

# Número total de páginas
NUM_PAGES = int(((soup.find('li', class_='next')).find_previous_sibling('li')).text.strip())

# Configurações de Driver
driver = webdriver.Chrome()
driver.get(URL)

# loop de págians
items = []

PAGE = 1
while PAGE <= NUM_PAGES:
    print(f"Processando página {PAGE} de {NUM_PAGES}")
    wait = WebDriverWait(driver, 10)
    
    # Aceitar cookies - Necessário pois o componente sobrepõe itens da página
    try:
        entendi_button = wait.until(EC.element_to_be_clickable((By.ID, "onetrust-accept-btn-handler")))
        entendi_button.click()
    except:
        pass
    
    wait.until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, 'article.productCard')))
    articles = driver.find_elements(By.CSS_SELECTOR, 'article.productCard')

    # URL.split('?')[0]: divide a URL pelo caractere '?' e pega a primeira parte antes dos parâmetros de consulta.
    # .split('/'): divide a parte restante da URL pelo caractere '/' 
    # [3]: Acessa o quarto elemento da lista, que é a categoria
    CATEGORIA = ((URL.split('?')[0]).split('/'))[3]
    
    # [-1]: Acessa o último elemento da lista, que é a subcategoria
    SUBCATEGORIA = ( ( (URL.split('?'))[0] ).split('/') )[-1]
    
    for article in articles:
        item_url = ('https://www.kabum.com.br/' + article.find_element(By.CSS_SELECTOR, 'a.productLink').get_attribute('href') )
        if item_url:
            item_url = item_url
        else:
            item_url = None 
            
        item_name = article.find_element(By.CSS_SELECTOR, 'h3').text
        item_image = article.find_element(By.CSS_SELECTOR, 'a.productLink > img').get_attribute('src')
        item_openBox = article.find_elements(By.CLASS_NAME, 'openboxTagCard')
        if item_openBox:
            item_openBox = True
        else:
            item_openBox = False
            
        item_price = article.find_element(By.CLASS_NAME, 'priceCard').text
        item_price = convert_price_to_float(item_price)   
            
        items.append({
            'categoria': CATEGORIA,
            'subcategoria': SUBCATEGORIA,
            'nome': item_name,
            'preço': item_price,
            'openBox': item_openBox,
            'imagem': item_image,
            'url': item_url,
        })
        
    try:
        next_button = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, 'li.next a.nextLink')))
        driver.execute_script("arguments[0].scrollIntoView(true);", next_button)
        driver.execute_script("arguments[0].click();", next_button)
        PAGE += 1
    except Exception as e:
        print("Erro ao clicar no botão próximo:", e)
        break
driver.quit()

Processando página 1 de 1


In [20]:
items

[{'categoria': 'hardware',
  'subcategoria': 'disco-rigido-hd',
  'nome': "HD Seagate 2TB BarraCuda, 3.5', SATA - ST2000DM008",
  'preço': 439.99,
  'openBox': False,
  'imagem': 'https://images.kabum.com.br/produtos/fotos/100916/100916_1552934430_index_m.jpg',
  'url': 'https://www.kabum.com.br/https://www.kabum.com.br/produto/100916/hd-seagate-2tb-barracuda-3-5-sata-st2000dm008'},
 {'categoria': 'hardware',
  'subcategoria': 'disco-rigido-hd',
  'nome': "HD Seagate 4TB BarraCuda, 3.5', SATA - ST4000DM004",
  'preço': 596.99,
  'openBox': False,
  'imagem': 'https://images.kabum.com.br/produtos/fotos/95803/95803_1522867513_index_m.jpg',
  'url': 'https://www.kabum.com.br/https://www.kabum.com.br/produto/95803/hd-seagate-4tb-barracuda-3-5-sata-st4000dm004'},
 {'categoria': 'hardware',
  'subcategoria': 'disco-rigido-hd',
  'nome': 'HD Western Digital Purple, 2TB, 64mb, Sata 3 - WD23PURZ',
  'preço': 429.99,
  'openBox': False,
  'imagem': 'https://images.kabum.com.br/produtos/fotos/472